In [1]:
import csv
import math
import warnings
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df_ads = pd.read_csv('gs://aes-datahub-0002-curated/Covid_Test/Test/res_cust_v6_07202020.csv')

In [3]:
df_ads.head()

,OPEN DATE,BALANCE,ACCT TYPE,CREDIT STATUS,COLLECTIONS HISTORY,PA TYPE,PA STATUS,ZIP,SERVICE CENTER,BILL UNIT,MASKED_ACCOUNT,m1_value,m2_value,m3_value,m4_value,m5_value,m6_value,m7_value,m8_value,m9_value,m10_value,m11_value,m12_value,CustScore,Acc_Disconnect_Count,Acc_Letter_Count,Acc_Due_Count_1,Acc_Due_Count_2,Acc_Due_Count_3,Acc_Personal_Contact_count,Acc_Balance_Flag,Acc_Due_Count,CUSTOMER,Covid,Onset,precovid,PA_Opted
0,7/19/1999,75.00,RES,A,110,NaN,,45159,WLM,15,1.619070e+19,0,0,0,0,0,0,0,0,0,1,1,0,2,0,0,2,0,0,0,1,2,37772.0,0,0,2,F
1,8/1/2000,8.32,RES,A,0,NaN,,45177,WLM,8,1.430650e+19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,49000.0,0,0,0,F
2,8/1/2000,8.49,RES,A,0,,,45177,WLM,8,1.338760e+19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,49000.0,0,0,0,F
3,8/1/2000,9.45,RES,A,0,,,45177,WLM,8,1.095920e+19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,49000.0,0,0,0,F
4,8/1/2000,9.88,RES,A,0,,,45177,WLM,8,8.208630e+18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,49000.0,0,0,0,F


In [10]:
print(list(df_ads.columns))

['OPEN DATE', 'BALANCE', 'ACCT TYPE', 'CREDIT STATUS', 'COLLECTIONS HISTORY', 'PA TYPE', 'PA STATUS', 'ZIP', 'SERVICE CENTER', 'BILL UNIT', 'MASKED_ACCOUNT', 'm1_value', 'm2_value', 'm3_value', 'm4_value', 'm5_value', 'm6_value', 'm7_value', 'm8_value', 'm9_value', 'm10_value', 'm11_value', 'm12_value', 'CustScore', 'Acc_Disconnect_Count', 'Acc_Letter_Count', 'Acc_Due_Count_1', 'Acc_Due_Count_2', 'Acc_Due_Count_3', 'Acc_Personal_Contact_count', 'Acc_Balance_Flag', 'Acc_Due_Count', 'CUSTOMER', 'Covid', 'Onset', 'precovid', 'PA_Opted']


In [12]:
ignore_features = ['OPEN DATE', 'ACCT TYPE', 'COLLECTIONS HISTORY', 'PA TYPE', 'PA STATUS', 'MASKED_ACCOUNT',
                   'm1_value', 'm2_value', 'm3_value', 'm4_value', 'm5_value', 'm6_value', 'm7_value',
                   'm8_value', 'm9_value', 'm10_value', 'm11_value', 'm12_value', 'CustScore', 'CUSTOMER',
                   'Acc_Balance_Flag']

df_ads.drop(ignore_features, axis=1, inplace=True)

In [13]:
print(list(df_ads.columns))

['BALANCE', 'CREDIT STATUS', 'ZIP', 'SERVICE CENTER', 'BILL UNIT', 'Acc_Disconnect_Count', 'Acc_Letter_Count', 'Acc_Due_Count_1', 'Acc_Due_Count_2', 'Acc_Due_Count_3', 'Acc_Personal_Contact_count', 'Acc_Due_Count', 'Covid', 'Onset', 'precovid', 'PA_Opted']


In [14]:
df_ads.head()

,BALANCE,CREDIT STATUS,ZIP,SERVICE CENTER,BILL UNIT,Acc_Disconnect_Count,Acc_Letter_Count,Acc_Due_Count_1,Acc_Due_Count_2,Acc_Due_Count_3,Acc_Personal_Contact_count,Acc_Due_Count,Covid,Onset,precovid,PA_Opted
0,75.00,A,45159,WLM,15,0,0,2,0,0,0,2,0,0,2,F
1,8.32,A,45177,WLM,8,0,0,0,0,0,0,0,0,0,0,F
2,8.49,A,45177,WLM,8,0,0,0,0,0,0,0,0,0,0,F
3,9.45,A,45177,WLM,8,0,0,0,0,0,0,0,0,0,0,F
4,9.88,A,45177,WLM,8,0,0,0,0,0,0,0,0,0,0,F


In [17]:
df_bill_unit = pd.get_dummies(df_ads['BILL UNIT'], prefix='BILL_UNIT', prefix_sep='_')

In [19]:
df_bill_unit.head()

,BILL_UNIT_1,BILL_UNIT_2,BILL_UNIT_3,BILL_UNIT_4,BILL_UNIT_5,BILL_UNIT_6,BILL_UNIT_7,BILL_UNIT_8,BILL_UNIT_9,BILL_UNIT_10,BILL_UNIT_11,BILL_UNIT_12,BILL_UNIT_13,BILL_UNIT_14,BILL_UNIT_15,BILL_UNIT_16,BILL_UNIT_17,BILL_UNIT_18,BILL_UNIT_19,BILL_UNIT_20,BILL_UNIT_21
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
df_ads.drop(['BILL UNIT'], axis=1, inplace=True)

In [21]:
df_ads_v1 = pd.merge(df_ads, df_bill_unit, left_index=True, right_index=True)

In [22]:
df_ads_v1.head()

,BALANCE,CREDIT STATUS,ZIP,SERVICE CENTER,Acc_Disconnect_Count,Acc_Letter_Count,Acc_Due_Count_1,Acc_Due_Count_2,Acc_Due_Count_3,Acc_Personal_Contact_count,Acc_Due_Count,Covid,Onset,precovid,PA_Opted,BILL_UNIT_1,BILL_UNIT_2,BILL_UNIT_3,BILL_UNIT_4,BILL_UNIT_5,BILL_UNIT_6,BILL_UNIT_7,BILL_UNIT_8,BILL_UNIT_9,BILL_UNIT_10,BILL_UNIT_11,BILL_UNIT_12,BILL_UNIT_13,BILL_UNIT_14,BILL_UNIT_15,BILL_UNIT_16,BILL_UNIT_17,BILL_UNIT_18,BILL_UNIT_19,BILL_UNIT_20,BILL_UNIT_21
0,75.00,A,45159,WLM,0,0,2,0,0,0,2,0,0,2,F,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,8.32,A,45177,WLM,0,0,0,0,0,0,0,0,0,0,F,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8.49,A,45177,WLM,0,0,0,0,0,0,0,0,0,0,F,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9.45,A,45177,WLM,0,0,0,0,0,0,0,0,0,0,F,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9.88,A,45177,WLM,0,0,0,0,0,0,0,0,0,0,F,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
df_ads_v1.dtypes

BALANCE                       float64
CREDIT STATUS                  object
ZIP                            object
SERVICE CENTER                 object
Acc_Disconnect_Count            int64
Acc_Letter_Count                int64
Acc_Due_Count_1                 int64
Acc_Due_Count_2                 int64
Acc_Due_Count_3                 int64
Acc_Personal_Contact_count      int64
Acc_Due_Count                   int64
Covid                           int64
Onset                           int64
precovid                        int64
PA_Opted                       object
BILL_UNIT_1                     uint8
BILL_UNIT_2                     uint8
BILL_UNIT_3                     uint8
BILL_UNIT_4                     uint8
BILL_UNIT_5                     uint8
BILL_UNIT_6                     uint8
BILL_UNIT_7                     uint8
BILL_UNIT_8                     uint8
BILL_UNIT_9                     uint8
BILL_UNIT_10                    uint8
BILL_UNIT_11                    uint8
BILL_UNIT_12

In [29]:
print(list(df_ads_v1['ZIP'].unique()))
print(df_ads_v1['ZIP'].nunique())

['45159', '45177', '43160', '45459', '45418', '45828', '45324', '45322', '45342', '43357', '45385', '45363', '45414', '45431', '45373', '45409', '45419', '45305', '45404', '45885', '45424', '45430', '45331', '43106', '45309', '45410', '43310', '43072', '45405', '45327', '45142', '45434', '45377', '45432', '45333', '45135', '45133', '45123', '45406', '45429', '43311', '45164', '45402', '45365', '45335', '45403', '45417', '45458', '45420', '43348', '45440', '45439', '45338', '45315', '45152', '45362', '45846', '45068', '43319', '45308', '45318', '43078', '45317', '43333', '45449', '45415', '43060', '45169', '45883', '45381', '45311', '45882', '45113', '43318', '45370', '43044', '45054', '45303', '43128', '43040', '43331', '45426', '45320', '45304', '43360', '45066', '45325', '45107', '45344', '45822', '45345', '43077', '45416', '45146', '45302', '45356', '45312', '43324', '45371', '45359', '45862', '45306', '45866', '45314', '45310', '43045', '45328', '43067', '45860', '43143', '43343', 

In [30]:
df_ads_v1['ZIP'] = df_ads_v1['ZIP'].astype(str)

In [31]:
df_ads_v1.dtypes

BALANCE                       float64
CREDIT STATUS                  object
ZIP                            object
SERVICE CENTER                 object
Acc_Disconnect_Count            int64
Acc_Letter_Count                int64
Acc_Due_Count_1                 int64
Acc_Due_Count_2                 int64
Acc_Due_Count_3                 int64
Acc_Personal_Contact_count      int64
Acc_Due_Count                   int64
Covid                           int64
Onset                           int64
precovid                        int64
PA_Opted                       object
BILL_UNIT_1                     uint8
BILL_UNIT_2                     uint8
BILL_UNIT_3                     uint8
BILL_UNIT_4                     uint8
BILL_UNIT_5                     uint8
BILL_UNIT_6                     uint8
BILL_UNIT_7                     uint8
BILL_UNIT_8                     uint8
BILL_UNIT_9                     uint8
BILL_UNIT_10                    uint8
BILL_UNIT_11                    uint8
BILL_UNIT_12

In [32]:
df_ads_v1.to_csv('gs://aes-datahub-0002-curated/Covid_Test/Test/res_cust_v6.1_07202020.csv')